In [41]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

In [42]:
nnfs.init()

In [43]:
class Layer_Dense:
    
    # Layer initialization
    def __init__(self, n_inputs, n_neurons):
        # Initialize weights and biases
        self.weights = 0.01 * np.random.randn(n_inputs, n_neurons)
        self.biases = np.zeros((1, n_neurons))
        
    # Forward pass
    def forward(self, inputs):
        # Calculate output values from inputs, weights and biases
        self.output = np.dot(inputs, self.weights) + self.biases
    
# ReLU Activation
class Activation_ReLU:
    
    #Forward pass
    def forward(self, inputs):
        #Calculate output values from inputs
        self.output = np.maximum(0, inputs)
        
# Softmax activation
class Activation_Softmax:
    
    #Forward pass
    def forward(self, inputs):
        
        #Get unnormalized probabilities
        exp_values = np.exp(inputs - np.max(inputs, axis=1, keepdims=True))
        
        #Normalize them for each sample
        probabilities = exp_values / np.sum(exp_values, axis =1, keepdims=True)
        
        self.output = probabilities
        
# Common loss class
class Loss:
    
    # Calculates the data and regularization losses
    # given model output and ground truth values
    def calculate(self, output, y):
        
        # Calculate sample losses
        sample_losses = self.forward(output, y)
        
        # Calculate mean loss
        data_loss = np.mean(sample_losses)
        
        #return loss
        return data_loss

# Cross-entropy loss
class Loss_CategoricalCrossEntropy(Loss):
    # Forward pass
    def forward(self, y_pred, y_true):
        
        # Number of samples in a batch
        samples = len(y_pred)
        
        # Clip data to prevent division by 0
        # Clip both sides to not drag mean towards any value
        # numpy.clip(a, a_min, a_max)
        y_pred_clipped = np.clip(y_pred, 1e-7, 1-1e-7)
        
        # Probablities for target values - 
        # only if categorical labels
        if len(y_true.shape) == 1:
            correct_confidences = y_pred_clipped[range(samples), y_true]
            
        # Mask values - only for one-hot  encoded labels
        elif len(y_true.shape) == 2:
            correct_confidences = np.sum(y_pred_clipped * y_true, axis=1)
            
        #Losses
        negative_log_likelihoods = -np.log(correct_confidences)
        return negative_log_likelihoods
 

        


In [44]:
# Create dataset
X, y = spiral_data(samples = 10, classes=3)

In [46]:
# Create Dense layer with 2 input features and 3 output values
dense1 = Layer_Dense(2,3)

In [48]:
#Create ReLU activation (to be used with Dense Layer)
activation1 = Activation_ReLU()

In [49]:
#  Create second Dense layer with 3 input  features (as we take output
# of previous layer here) and 3 output values
dense2 = Layer_Dense(3,3)

In [50]:
# Create Softmax activation (to be used with Dense Layer):
activation2 = Activation_Softmax()

In [51]:
# Create loss function
loss_function = Loss_CategoricalCrossEntropy()

In [52]:
# Perform a forward pass of our training data through this layer
dense1.forward(X)

In [53]:
# Perform a forward pass through activation function
# it takes the output of fisrt dense layer here
activation1.forward(dense1.output)

In [54]:
# Perform a forward pass through second Dense layer
# it takes outputs of activation function of first layer as inputs
dense2.forward(activation1.output)

In [55]:
# Perform a forward pass through activation function
# it takes the output of second dense layer here
activation2.forward(dense2.output)

In [56]:
# Let's see output of the first few samples
print(activation2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.33333397 0.33333322 0.3333328 ]
 [0.3333431  0.33334008 0.3333168 ]
 [0.3333208  0.33335394 0.33332527]
 [0.33331427 0.33335733 0.33332837]]


In [58]:
# Perform a forward pass through loss function
#it takes the output of second dense layer here and returns loss
loss = loss_function.calculate(activation2.output, y)

In [59]:
print('loss: {}'.format(loss))

loss: 1.0986138582229614


In [64]:
# Calcultae accuracy from output of activation2 and targets
# calculate values along first axis
predictions = np.argmax(activation2.output, axis=1)
if len(y.shape) == 2:
    y = np.argmax(y, axis=1)
accuracy = np.mean(predictions==y)
print('acc : {}'.format(accuracy))

acc : 0.3333333333333333


In [61]:
predictions

array([0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0,
       1, 1, 0, 1, 0, 0, 1, 1], dtype=int64)